In [9]:
import keras
from keras.datasets import cifar10
import numpy as np
from keras.applications.inception_v3 import InceptionV3, preprocess_input
import scipy
from scipy import misc
import os

# load the data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = np.squeeze(y_train)
print('data loaded')

data loaded


In [10]:
# load Resnet50 model + remove final classification layers

from keras import applications

model = applications.ResNet50(include_top=False, weights='imagenet')

print('model loaded')

/anaconda3/envs/tfdeeplearning/lib/python3.5/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


model loaded


In [11]:
# obtain bottleneck features (train)
if os.path.exists('resnet50_features_train.npz'):
    print('bottleneck features detected (train)')
    features = np.load('resnet50_features_train.npz')['features']
else:
    print('bottleneck features file not detected (train)')
    print('calculating now ...')
    # pre-process the train data
    big_x_train = np.array([scipy.misc.imresize(x_train[i], (139, 139, 3)) 
                            for i in range(0, len(x_train))]).astype('float32')
    resnet_input_train = preprocess_input(big_x_train)
    print('train data preprocessed')
    # extract, process, and save bottleneck features
    features = model.predict(resnet_input_train)
    features = np.squeeze(features)
    np.savez('resnet50_features_train', features=features)
print('bottleneck features saved (train)')

bottleneck features detected (train)
bottleneck features saved (train)


In [12]:
# obtain bottleneck features (test)
if os.path.exists('resnet_features_test.npz'):
    print('bottleneck features detected (test)')
    features_test = np.load('resnet_features_test.npz')['features_test']
else:
    print('bottleneck features file not detected (test)')
    print('calculating now ...')
    # pre-process the test data
    big_x_test = np.array([scipy.misc.imresize(x_test[i], (139, 139, 3)) 
                       for i in range(0, len(x_test))]).astype('float32')
    resnet_input_test = preprocess_input(big_x_test)
    # extract, process, and save bottleneck features (test)
    features_test = model.predict(resnet_input_test)
    features_test = np.squeeze(features_test)
    np.savez('resnet_features_test', features_test=features_test)
print('bottleneck features saved (test)')

bottleneck features detected (test)
bottleneck features saved (test)


In [13]:
from keras.utils import np_utils

# one-hot encode the labels
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [14]:
from keras.callbacks import ModelCheckpoint   
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPooling2D, BatchNormalization

model = Sequential()
# model.add(Conv2D(filters=100, kernel_size=2, input_shape=features.shape[1:]))
model.add(Dense(512, activation='relu', input_shape=features.shape[1:]))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5, 5, 512)         1049088   
_________________________________________________________________
dropout_3 (Dropout)          (None, 5, 5, 512)         0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 5, 5, 512)         2048      
_________________________________________________________________
flatten_2 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               3277056   
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 256)               1024      
__________

In [15]:
# optimizer = keras.optimizers.rmsprop(lr=0.0003,decay=1e-6)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='model.best.hdf5', verbose=1, save_best_only=True)

history = model.fit(features, y_train, batch_size=64, epochs=20,
          validation_split=0.1, callbacks=[checkpointer],
          verbose=2, shuffle=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/20
 - 73s - loss: 1.9873 - acc: 0.2610 - val_loss: 1.7579 - val_acc: 0.3468

Epoch 00001: val_loss improved from inf to 1.75792, saving model to model.best.hdf5
Epoch 2/20
 - 59s - loss: 1.7835 - acc: 0.3424 - val_loss: 1.7445 - val_acc: 0.3694

Epoch 00002: val_loss improved from 1.75792 to 1.74449, saving model to model.best.hdf5
Epoch 3/20
 - 58s - loss: 1.7076 - acc: 0.3793 - val_loss: 1.5821 - val_acc: 0.4214

Epoch 00003: val_loss improved from 1.74449 to 1.58206, saving model to model.best.hdf5
Epoch 4/20
 - 58s - loss: 1.6389 - acc: 0.4084 - val_loss: 1.5655 - val_acc: 0.4100

Epoch 00004: val_loss improved from 1.58206 to 1.56555, saving model to model.best.hdf5
Epoch 5/20
 - 57s - loss: 1.5998 - acc: 0.4209 - val_loss: 1.5120 - val_acc: 0.4382

Epoch 00005: val_loss improved from 1.56555 to 1.51198, saving model to model.best.hdf5
Epoch 6/20
 - 57s - loss: 1.5759 - acc: 0.4357 - val_loss: 1.5421 - val_acc: 0.4320

Epoch

In [16]:
# load the weights that yielded the best validation accuracy
model.load_weights('model.best.hdf5')

# evaluate test accuracy
score = model.evaluate(features_test, y_test, verbose=)
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy: %.4f%%' % accuracy)

SyntaxError: invalid syntax (<ipython-input-16-88a585a45d13>, line 5)

In [ ]:
from matplotlib import pyplot
%matplotlib inline

# plot learning curves of model accuracy
pyplot.plot(history.history['acc'], label='train')
pyplot.plot(history.history['val_acc'], label='test')
pyplot.legend()
pyplot.show()